In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import pickle
import networkx as nx
from scipy.stats.mstats import gmean
from scipy.stats import spearmanr, ks_2samp
from scipy.signal import find_peaks, peak_prominences
from load_files_pop import (
    Ref,
    Z_20,
    Ref_pop20,
    Z_200,
    Ref_pop200,
    Z_2000,
    Ref_pop2000
)
from Reference_pop import simulate_reference_time_series
from plot_search_results import (
    plot_pareto_front3D,
    plot_pulse_ensemble_time_series,
    plot_ref_ensemble_time_series
)
from amplifier_problem import Amplifier
from signal_conditioner_problem import SignalConditioner

orange_ = [i/255 for i in [230, 159, 0]]
sky_blue = [i/255 for i in [86, 180, 233]]
pink_ = [i/255 for i in [204, 121, 167]]
bluish_green = [i/255 for i in [0, 158, 115]]
vermillion = [i/255 for i in [213, 94, 0]]
yellow_ = [i/255 for i in [240, 228, 66]]
blue_ = [i/255 for i in [0, 114, 178]]

colors = [orange_, sky_blue, pink_, bluish_green, vermillion, yellow_, blue_]

Z_200_df = pd.DataFrame(data = Z_200, columns = ["plasmid_" + str(i) for i in range(9)])
Z_200_df = Z_200_df.drop(labels=["plasmid_" + str(i) for i in range(5,9)], axis=1)
Z_200_df["average"] = np.mean(Z_200_df, axis=1)

# Pulse test case

### Labeling topologies on pareto front- single cell model

In [ ]:
def sort_in_dict(in_dict):
    sorted_in_dict = {}
    for key, val in in_dict.items():
    # print(key, val)
        val_sorted = {}
        for subkey, subval in val.items():
            subval.sort()
        #     print(subkey, subval)
            val_sorted.update({subkey: subval})
        sorted_in_dict.update({key: val_sorted})
    return sorted_in_dict


def generalize_in_dict(in_dict):
    in_dict_general = {}
    # format of keys: "part|regulator1|regulator2|..."
    for key, value in in_dict.items():
        i = 0
        key_reg = key[0]
        if len(value["P"]) > 0:
            key_reg += "P"
        if len(value["Z"]) > 0:
            key_regZ = ""
            for Zreg in value["Z"]:
                if Zreg == key:
                    key_regZ += "A"
                else:
                    key_regZ += "Z"
            key_regZ = "".join(sorted(key_regZ))
            key_reg += key_regZ
        if len(value["I"]) > 0:
            key_regI = ""
            for Ireg in value["I"]:
                if Ireg == key:
                    key_regI += "A"
                else:
                    key_regI += "I"
            key_regI = "".join(sorted(key_regI))
            key_reg += key_regI
        in_dict_general.update({key_reg: value})
    
    return in_dict_general

In [ ]:
path_pulse = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Pulse_single_cell/Optimized_hyperparams/t_pulse_fixed_pop_max_hv/run2_ngen10000/2025-03-13_Pulse_single_DsRED_t_pulse_opt_hps_ngen100_seed_6/2025-03-24_results_analysis/"
fname = "selected_results_full_pareto.pkl"
pulse_pareto = pd.read_pickle(path_pulse+fname)

objs0 = []
in_dict0 = {'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'I13': {'P': [], 'Z': ['Z6'], 'I': []}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': []}}

sorted_in_dict0 = sort_in_dict(in_dict0)

objs1 = []
in_dict1 = {'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'I13': {'P': ['P1'], 'Z': ['Z6'], 'I': []}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': []}}

sorted_in_dict1 = sort_in_dict(in_dict1)

objs2 = []
in_dict2 = {'I13': {'P': [], 'Z': ['Z6'], 'I': []}, 'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': ['I13']}}

sorted_in_dict2 = sort_in_dict(in_dict2)

objs3 = []
in_dict3 = {'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I2']}, 'I2': {'P': ['P1'], 'Z': ['Z6'], 'I': []}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': ['I2']}}

sorted_in_dict3 = sort_in_dict(in_dict3)


in_dict0_general = generalize_in_dict(in_dict0)
# print(in_dict0_general.keys())
in_dict1_general = generalize_in_dict(in_dict1)
# print(in_dict1_general.keys())
in_dict2_general = generalize_in_dict(in_dict2)
# print(in_dict2_general.keys())
in_dict3_general = generalize_in_dict(in_dict3)
# print(in_dict3_general.keys())


dict_list = [sorted_in_dict0, sorted_in_dict1, sorted_in_dict2, sorted_in_dict3]
dict_list_general = [in_dict0_general, in_dict1_general, in_dict2_general, in_dict3_general]
keys_list = [d.keys() for d in dict_list_general]


opt_circuits = {
    "opt_0_topology": [],
    "opt_0_t_pulse": [],
    "opt_0_prom_rel": [],
    "opt_0_doses": [],

    "opt_1_topology": [],
    "opt_1_t_pulse": [],
    "opt_1_prom_rel": [],
    "opt_1_doses": [],
    
    "opt_2_topology": [],
    "opt_2_t_pulse": [],
    "opt_2_prom_rel": [],
    "opt_2_doses": [],

    "opt_3_topology": [],
    "opt_3_t_pulse": [],
    "opt_3_prom_rel": [],
    "opt_3_doses": [],
}
opt_variants = {
    "opt_0_var_topology": [],
    "opt_0_var_t_pulse": [],
    "opt_0_var_prom_rel": [],
    "opt_0_var_doses": [],

    "opt_1_var_topology": [],
    "opt_1_var_t_pulse": [],
    "opt_1_var_prom_rel": [],
    "opt_1_var_doses": [],

    "opt_2_var_topology": [],
    "opt_2_var_t_pulse": [],
    "opt_2_var_prom_rel": [],
    "opt_2_var_doses": [],

    "opt_3_var_topology": [],
    "opt_3_var_t_pulse": [],
    "opt_3_var_prom_rel": [],
    "opt_3_var_doses": [],
}

for index, row in pulse_pareto.iterrows():
    # print("CI circuit ", index)
    row_in_dict = row["Parts order"]
    sorted_in_dict = sort_in_dict(row_in_dict)
    row_in_dict_general = generalize_in_dict(row_in_dict)
    if sorted_in_dict in dict_list:
        opt_circuit_idx = keys_list.index(row_in_dict_general.keys())
        # print("circuit is opt ", opt_circuit_idx)
        opt_circuits["opt_"+str(opt_circuit_idx)+"_topology"].append(row["Topology"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_t_pulse"].append(row["t_pulse"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_prom_rel"].append(row["prominence_rel"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_doses"].append(row["Topology"].dose)
    elif row_in_dict_general.keys() in keys_list:
        variant_idx = keys_list.index(row_in_dict_general.keys())
        # print("circuit is variant of opt ", keys_list.index(row_in_dict_general.keys()))
        opt_variants["opt_"+str(variant_idx)+"_var_topology"].append(row["Topology"])
        opt_variants["opt_"+str(variant_idx)+"_var_t_pulse"].append(row["t_pulse"])
        opt_variants["opt_"+str(variant_idx)+"_var_prom_rel"].append(row["prominence_rel"])
        opt_variants["opt_"+str(variant_idx)+"_var_doses"].append(row["Topology"].dose)


    # print("*****")
opt_circuits_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_circuits.items()]))
opt_variants_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_variants.items()]))
display(opt_circuits_df)
opt_circuits_df_nonan = opt_circuits_df.fillna(value="", inplace=False)
opt_circuits_df.to_csv(path_pulse+"labeled_pareto_circuits.csv")
opt_variants_df.to_csv(path_pulse+"labeled_pareto_circuit_variants.csv")

In [ ]:
# path_pulse = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Pulse_single_cell/Optimized_hyperparams/t_pulse/Optimized_hyperparams_fixed_pop_max_hv/2024-10-09_Pulse_single_DsRED_t_pulse_opt_hps_ngen80_seed_5/2024-10-11_results_analysis/"
# fname = "selected_results_full_pareto.pkl"
# pulse_pareto = pd.read_pickle(path_pulse+fname)

# objs0 = []
# in_dict0 = {'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'I13': {'P': [], 'Z': ['Z6'], 'I': []}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': []}}

# sorted_in_dict0 = sort_in_dict(in_dict0)

# objs1 = []
# in_dict1 = {'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'I13': {'P': ['P1'], 'Z': ['Z6'], 'I': []}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': []}}

# sorted_in_dict1 = sort_in_dict(in_dict1)

# objs2 = []
# in_dict2 = {'I13': {'P': [], 'Z': ['Z6'], 'I': []}, 'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': ['I13']}}

# sorted_in_dict2 = sort_in_dict(in_dict2)




# in_dict0_general = generalize_in_dict(in_dict0)
# # print(in_dict0_general.keys())
# in_dict1_general = generalize_in_dict(in_dict1)
# # print(in_dict1_general.keys())
# in_dict2_general = generalize_in_dict(in_dict2)
# print(in_dict2_general.keys())

# # print(in_dict5_general.keys())
# dict_list = [sorted_in_dict0, sorted_in_dict1, sorted_in_dict2]
# dict_list_general = [in_dict0_general, in_dict1_general, in_dict2_general]
# keys_list = [d.keys() for d in dict_list_general]


# opt_circuits = {
#     "opt_0_topology": [],
#     "opt_0_t_pulse": [],
#     "opt_0_prom_rel": [],
#     "opt_0_doses": [],

#     "opt_1_topology": [],
#     "opt_1_t_pulse": [],
#     "opt_1_prom_rel": [],
#     "opt_1_doses": [],
    
#     "opt_2_topology": [],
#     "opt_2_t_pulse": [],
#     "opt_2_prom_rel": [],
#     "opt_2_doses": [],
# }
# opt_variants = {
#     "opt_0_var_topology": [],
#     "opt_0_var_t_pulse": [],
#     "opt_0_var_prom_rel": [],
#     "opt_0_var_doses": [],

#     "opt_1_var_topology": [],
#     "opt_1_var_t_pulse": [],
#     "opt_1_var_prom_rel": [],
#     "opt_1_var_doses": [],


#     "opt_2_var_topology": [],
#     "opt_2_var_t_pulse": [],
#     "opt_2_var_prom_rel": [],
#     "opt_2_var_doses": [],
# }

# for index, row in pulse_pareto.iterrows():
#     # print("CI circuit ", index)
#     row_in_dict = row["Parts order"]
#     sorted_in_dict = sort_in_dict(row_in_dict)
#     row_in_dict_general = generalize_in_dict(row_in_dict)
#     if sorted_in_dict in dict_list:
#         opt_circuit_idx = keys_list.index(row_in_dict_general.keys())
#         # print("circuit is opt ", opt_circuit_idx)
#         opt_circuits["opt_"+str(opt_circuit_idx)+"_topology"].append(row["Topology"])
#         opt_circuits["opt_"+str(opt_circuit_idx)+"_t_pulse"].append(row["t_pulse"])
#         opt_circuits["opt_"+str(opt_circuit_idx)+"_prom_rel"].append(row["prominence_rel"])
#         opt_circuits["opt_"+str(opt_circuit_idx)+"_doses"].append(row["Topology"].dose)
#     elif row_in_dict_general.keys() in keys_list:
#         variant_idx = keys_list.index(row_in_dict_general.keys())
#         # print("circuit is variant of opt ", keys_list.index(row_in_dict_general.keys()))
#         opt_variants["opt_"+str(variant_idx)+"_var_topology"].append(row["Topology"])
#         opt_variants["opt_"+str(variant_idx)+"_var_t_pulse"].append(row["t_pulse"])
#         opt_variants["opt_"+str(variant_idx)+"_var_prom_rel"].append(row["prominence_rel"])
#         opt_variants["opt_"+str(variant_idx)+"_var_doses"].append(row["Topology"].dose)


#     # print("*****")
# opt_circuits_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_circuits.items()]))
# opt_variants_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_variants.items()]))
# display(opt_circuits_df)
# opt_circuits_df_nonan = opt_circuits_df.fillna(value="", inplace=False)
# opt_circuits_df.to_csv(path_pulse+"labeled_pareto_circuits.csv")
# opt_variants_df.to_csv(path_pulse+"labeled_pareto_circuit_variants.csv")

### Labeling topologies on pareto front- pop20 frac_pulse

In [ ]:
# 20-cell model
path_pulse = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/Pulse_seed_pop_DsRED_inhibitor/frac_pulse/2024-10-13_Pulse_pop_DsRED_3obj_frac_pulse_ngen50_seed_0/"
fname0 = "2024-10-16_results_analysis_low_fracp/selected_results_low_frac_p.pkl"
fname1 = "2024-10-13_results_analysis/selected_results_frac_pulse.pkl"

# 200-cell model
# path_pulse = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/Pulse_seed_pop_DsRED_inhibitor/frac_pulse/Pulse_pop_DsRED_inhibitor_3obj_frac_pulse_pop200_ngen100/2024-10-13_Pulse_pop_DsRED_inhibitor_3obj_frac_pulse_pop200_ngen100/2024-10-13_results_analysis/"
# fname0 = "0_025/selected_results_low_frac_p.pkl"
# fname1 = "025_03/selected_results_mid_pulse.pkl"
# fname2 = "03_04/selected_results_mid_pulse.pkl"
# fname3 = "04_05/selected_results_mid_pulse.pkl"
# fname4 = "05_06/selected_results_mid_pulse.pkl"
# fname5 = "06_07/selected_results_mid_pulse.pkl"
# fname6 = "07_08/selected_results_mid_pulse.pkl"

pulse_pareto0 = pd.read_pickle(path_pulse+fname0)
pulse_pareto1 = pd.read_pickle(path_pulse+fname1)
# pulse_pareto2 = pd.read_pickle(path_pulse+fname2)
# pulse_pareto3 = pd.read_pickle(path_pulse+fname3)
# pulse_pareto4 = pd.read_pickle(path_pulse+fname4)
# pulse_pareto5 = pd.read_pickle(path_pulse+fname5)
# pulse_pareto6 = pd.read_pickle(path_pulse+fname6)

# df_full_pareto = pd.concat([pulse_pareto0, pulse_pareto1, pulse_pareto2, pulse_pareto3, pulse_pareto4, pulse_pareto5, pulse_pareto6], axis=0)
df_full_pareto = pd.concat([pulse_pareto0, pulse_pareto1], axis=0)
# display(df_full_pareto)

objs0 = []
in_dict0 = {'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'I13': {'P': [], 'Z': ['Z6'], 'I': []}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': []}}

sorted_in_dict0 = sort_in_dict(in_dict0)

objs1 = []
in_dict1 = {'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'I13': {'P': ['P1'], 'Z': ['Z6'], 'I': []}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': []}}

sorted_in_dict1 = sort_in_dict(in_dict1)

objs2 = []
in_dict2 = {'I13': {'P': [], 'Z': ['Z6'], 'I': []}, 'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': ['I13']}}

sorted_in_dict2 = sort_in_dict(in_dict2)

objs3 = []
in_dict3 = {'Z6': {'P': ['P1'], 'Z': ['Z6'], 'I': ['I13']}, 'I13': {'P': ['P1'], 'Z': ['Z6'], 'I': []}, 'Rep': {'P': [], 'Z': ['Z6'], 'I': ['I13']}}

sorted_in_dict3 = sort_in_dict(in_dict3)


in_dict0_general = generalize_in_dict(in_dict0)
# print(in_dict0_general.keys())
in_dict1_general = generalize_in_dict(in_dict1)
# print(in_dict1_general.keys())
in_dict2_general = generalize_in_dict(in_dict2)
# print(in_dict2_general.keys())
in_dict3_general = generalize_in_dict(in_dict3)
# print(in_dict3_general.keys())

# print(in_dict5_general.keys())
dict_list = [sorted_in_dict0, sorted_in_dict1, sorted_in_dict2, sorted_in_dict3]
dict_list_general = [in_dict0_general, in_dict1_general, in_dict2_general, in_dict3_general]
keys_list = [d.keys() for d in dict_list_general]

opt_circuits = {
    "opt_0_topology": [],
    "opt_0_frac_pulse": [],
    "opt_0_t_pulse": [],
    "opt_0_prom_rel": [],
    "opt_0_doses": [],

    "opt_1_topology": [],
    "opt_1_frac_pulse": [],
    "opt_1_t_pulse": [],
    "opt_1_prom_rel": [],
    "opt_1_doses": [],
    
    "opt_2_topology": [],
    "opt_2_frac_pulse": [],
    "opt_2_t_pulse": [],
    "opt_2_prom_rel": [],
    "opt_2_doses": [],

    "opt_3_topology": [],
    "opt_3_frac_pulse": [],
    "opt_3_t_pulse": [],
    "opt_3_prom_rel": [],
    "opt_3_doses": [],
}
opt_variants = {
    "opt_0_var_topology": [],
    "opt_0_var_frac_pulse": [],
    "opt_0_var_t_pulse": [],
    "opt_0_var_prom_rel": [],
    "opt_0_var_doses": [],

    "opt_1_var_topology": [],
    "opt_1_var_frac_pulse": [],
    "opt_1_var_t_pulse": [],
    "opt_1_var_prom_rel": [],
    "opt_1_var_doses": [],


    "opt_2_var_topology": [],
    "opt_2_var_frac_pulse": [],
    "opt_2_var_t_pulse": [],
    "opt_2_var_prom_rel": [],
    "opt_2_var_doses": [],

    "opt_3_var_topology": [],
    "opt_3_var_frac_pulse": [],
    "opt_3_var_t_pulse": [],
    "opt_3_var_prom_rel": [],
    "opt_3_var_doses": [],
}

for index, row in df_full_pareto.iterrows():
    # print("CI circuit ", index)
    row_in_dict = row["Parts order"]
    sorted_in_dict = sort_in_dict(row_in_dict)
    row_in_dict_general = generalize_in_dict(row_in_dict)
    if sorted_in_dict in dict_list:
        opt_circuit_idx = keys_list.index(row_in_dict_general.keys())
        # print("circuit is opt ", opt_circuit_idx)
        opt_circuits["opt_"+str(opt_circuit_idx)+"_topology"].append(row["Topology"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_frac_pulse"].append(row["frac_pulse"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_t_pulse"].append(row["t_pulse"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_prom_rel"].append(row["prominence_rel"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_doses"].append(row["Topology"].dose)
    elif row_in_dict_general.keys() in keys_list:
        variant_idx = keys_list.index(row_in_dict_general.keys())
        # print("circuit is variant of opt ", keys_list.index(row_in_dict_general.keys()))
        opt_variants["opt_"+str(variant_idx)+"_var_topology"].append(row["Topology"])
        opt_variants["opt_"+str(variant_idx)+"_var_frac_pulse"].append(row["frac_pulse"])
        opt_variants["opt_"+str(variant_idx)+"_var_t_pulse"].append(row["t_pulse"])
        opt_variants["opt_"+str(variant_idx)+"_var_prom_rel"].append(row["prominence_rel"])
        opt_variants["opt_"+str(variant_idx)+"_var_doses"].append(row["Topology"].dose)


#     # print("*****")
opt_circuits_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_circuits.items()]))
opt_variants_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_variants.items()]))
display(opt_circuits_df)
display(opt_variants_df)
# opt_circuits_df_nonan = opt_circuits_df.fillna(value="", inplace=False)
opt_circuits_df.to_csv(path_pulse+"labeled_CI_circuits_TEST.csv")
opt_variants_df.to_csv(path_pulse+"labeled_CI_circuit_variants_TEST.csv")

# print(opt_circuits["opt_3_topology"])

### Spline fitting

In [ ]:
def fit_spline_condition(all_cell_list):

    time_points = [14, 18, 22, 26, 38, 42, 46]
    pulse_rel_all_cells = []
    for i in range(len(all_cell_list)):
        pulse_rep_rel = all_cell_list[i]
        rep_rel_t_exp = [pulse_rep_rel[t] for t in time_points]
        pulse_rel_all_cells.append(rep_rel_t_exp)
    pulse_rel_t_exp_df = pd.DataFrame(pulse_rel_all_cells, columns = time_points)
    pulse_rel_t_exp_df_T = pulse_rel_t_exp_df.transpose().copy()
    pulse_rel_t_exp_df_T["Time (h)"] = pulse_rel_t_exp_df_T.index
    pulse_rel_t_exp_df_T_plot = pd.melt(frame=pulse_rel_t_exp_df_T,
                                      id_vars="Time (h)",
                                      var_name="column_name",
                                      value_name="Rep_rel")
    pulse_rel_t_exp_df_T_plot_sorted = pulse_rel_t_exp_df_T_plot.sort_values("Time (h)")
    spline_condition = np.polyfit(pulse_rel_t_exp_df_T_plot_sorted["Time (h)"].tolist(), pulse_rel_t_exp_df_T_plot_sorted["Rep_rel"].tolist(), deg=3.5)
    poly_spline = np.poly1d(spline_condition)
    spline_derivative = poly_spline.deriv()

    return poly_spline, spline_derivative, pulse_rel_t_exp_df_T_plot
 

In [ ]:
def plot_spline_condition(
      figure_path,
      poly_spline,
      poly_spline_all,
      condition,
      data_points=False,
      pulse_rel_t_exp_df_T_plot=None
):
   fig, ax = plt.subplots(1, 1, figsize=(1.65, 1.65))
   times = np.arange(0, 47, .1)
   spline_eval = poly_spline(times)
   ax.plot(times, spline_eval, color="grey")
   ax.set_xticks(np.arange(14, 47, 4))
   ax.set_ylim(bottom=0)
   ax.set_yticks([0, 2, 4, 6, 8])
   ax.set_xlabel("time (h)")
   ax.set_ylabel("Rep_rel fit")
   ax.set_title(condition)
   if data_points:
      sns.scatterplot(data=pulse_rel_t_exp_df_T_plot, x="Time (h)", y="Rep_rel", ax=ax, s=10, color="grey")
   # # plt.legend()
   # plt.show()
   plt.savefig("/Users/kdreyer/Desktop/pulse_200_cell_spline"+condition[-1])

In [ ]:
def plot_spline_derivative_condition(figure_path, spline_derivative,
                                     spline_derivative_all, condition):
   fig, ax = plt.subplots(1, 1, figsize=(1.65, 1.65))
   times = np.arange(14, 47, .1)
   times_exp = [14, 18, 22, 26, 38, 42, 46]
   derivative_eval = spline_derivative(times)
   derivative_eval_exp = spline_derivative(times_exp)
   print(derivative_eval_exp)
   # pct_change_derivative = [((derivative_eval_exp[i]-derivative_eval_exp[i-1])/derivative_eval_exp[i])*100 for i in range(1, len(derivative_eval_exp))]
   # print(condition, " % change derivative: ", pct_change_derivative)
   # ax.plot(times_exp, derivative_eval_exp, linestyle="none", marker="o", color="grey")
   ax.plot(times, derivative_eval, color="grey")
   ax.set_xticks(np.arange(14, 47, 4))
   ax.set_yticks([0, 0.05, 0.1])
   ax.set_ylim(top=0.12)
   ax.set_xlabel("time (h)")
   ax.set_ylabel("polynomial fit derivative")
   ax.set_title(condition)
   # plt.show()
   plt.savefig("/Users/kdreyer/Desktop/pulse_200_cell_deriv"+condition[-1])
    

In [ ]:
repo_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/"
pulse_exp = "Pulse_pop/Experiment_synTF1&2/for_paper/"

# Ref_all_cells = simulate_reference_time_series(["P1"], Z_200)
# ref_on = Ref_pop200["P1"]["on"]
# ref_all_cell_time_series = Ref_all_cells["P1"]["on all cells"]
# ref_rep_rel_all_cell = [ref_time_series/ref_on for ref_time_series in ref_all_cell_time_series]
# reference_ts_df = pd.DataFrame(index=["reference"], columns=["Topology",
#                                         "Rep_rel time series for each cell"], dtype=object)
# reference_ts_df["Topology"] = ["reference_topo"]
# reference_ts_df["Rep_rel time series for each cell"] = [ref_rep_rel_all_cell]
# reference_ts_df["condition"] = ["reference"]
# display(reference_ts_df)
# reference_ts_df.to_pickle(repo_path+pulse_exp+"200-cell_all_cells_ref.pkl")

repo_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/"
path_pulses = "Pulse_pop/Experiment_synTF1&2/200_cell_all_cell_results_df.pkl"
ref_fname = "Pulse_pop/Experiment_synTF1&2/for_paper/200-cell_all_cells_ref.pkl"

all_cell_results_df = pd.read_pickle(repo_path+path_pulses)
all_cell_ref_df = pd.read_pickle(repo_path+ref_fname)
# display(all_cell_results_df)
times = np.arange(0, 127, 1)
for index, row in all_cell_results_df.iterrows():
    ts_mean = row["Rep_rel time series mean"]
    plt.plot(times, ts_mean)
    plt.show()

selected_paper = all_cell_results_df.iloc[[0, 2, 3, 4]]
selected_paper_ts = selected_paper.drop(["Rep_rel time series mean",
                                         "single_cell_peaks", "single_cell_prominence"], axis=1)
selected_paper_ts.index=["pulse1", "pulse3", "pulse4", "pulse5"]
# display(selected_paper_ts)

all_conditions_ts = pd.concat([selected_paper_ts, all_cell_ref_df])
# display(all_conditions_ts)
# all_conditions_ts.to_pickle(repo_path+"Pulse_pop/Experiment_synTF1&2/for_paper/200-cell_all_cells_exp.pkl")


In [ ]:

plasmid_uptake_list = Z_200_df["average"].tolist()
plasmid_uptake_arr = np.array(plasmid_uptake_list)
max_uptake = max(Z_200_df["average"])
threshold_percent = 70
threshold_frac = 0.7
uptake_min = max_uptake*threshold_frac
idx_uptake_included = np.where(plasmid_uptake_arr >= uptake_min)[0]

repo_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/"
path_pulses = "Pulse_pop/Experiment_synTF1&2/200_cell_all_cell_results_df.pkl"
path_save = repo_path + "Pulse_pop/Experiment_synTF1&2/"

all_cell_results_df = pd.read_pickle(repo_path+path_pulses)
# display(all_cell_results_df)

# conditions = ["Pulse 1", "Pulse 2", "Pulse 4", "Pulse 6", "Pulse 5"]
drop_labels = ["Topology","Rep_rel time series mean"]

all_cell_time_series_opt = all_cell_results_df.drop(drop_labels, axis=1)

spline_deriv_pulse_list = []
poly_spline_pulse_list = []

spline_deriv_all_list = []
poly_spline_all_list = []

spline_refp_list = []
spline_refp_deriv_list = []

spline_ref_all_list = []
spline_ref_all_deriv_list = []

for index, row in all_cell_time_series_opt.iterrows():
        row_as_list = all_cell_time_series_opt.loc[index].tolist()
        threshold_cell_ts = [row_as_list[0][i] for i in idx_uptake_included]
        # for ts in threshold_cell_ts:
        #         plt.plot(np.arange(0, 127, 1), ts)
        # plt.show()
        # (poly_spline_pulse,
        #  spline_derivative_pulse,
        #  pulse_rel_t_exp_df_T_plot) = fit_spline_condition(threshold_cell_ts)
        
        # # print(poly_spline)
        # times = np.arange(14, 47, .1)
        # spline_eval_pulse = poly_spline_pulse(times)
        # # plt.plot(times, spline_eval)
        # poly_spline_pulse_list.append(spline_eval_pulse)
        # derivative_eval_pulse = spline_derivative_pulse(times)
        # spline_deriv_pulse_list.append(derivative_eval_pulse)

        # plot_spline_condition("", poly_spline_pulse, "Pulse "+str(index+1), 
        #                       data_points=True, 
        #                       pulse_rel_t_exp_df_T_plot=pulse_rel_t_exp_df_T_plot)
        # plot_spline_derivative_condition("", spline_derivative_pulse, "Pulse "+str(index+1))


        ###all cells
        # (poly_spline_all,
        #  spline_derivative_all,
        #  all_rel_t_exp_df_T_plot) = fit_spline_condition(row_as_list[0])
        
        # # print(poly_spline)
        # times = np.arange(14, 47, .1)
        # spline_eval_all = poly_spline_all(times)
        # # plt.plot(times, spline_eval)
        # poly_spline_all_list.append(spline_eval_all)
        # derivative_eval_all = spline_derivative_all(times)
        # spline_deriv_all_list.append(derivative_eval_all)

        # plot_spline_condition("", poly_spline_all, "All cell "+str(index+1), 
        #                       data_points=True, 
        #                       pulse_rel_t_exp_df_T_plot=all_rel_t_exp_df_T_plot)
        # plot_spline_derivative_condition("", spline_derivative_all, "All cell "+str(index+1))




        # plot_pulse_ensemble_time_series(repo_path+path_save, row_as_list, "Pulse "+str(index+1))

# plot_ref_ensemble_time_series(repo_path+save_path, ref_pulse_cells_time_series)
# plot_ref_ensemble_violin(repo_path+save_path, ref_pulse_cells_time_series)
# plot_ref_ensemble_time_series("", ref_all_cell_time_series)
ref_pulse_time_series = [ref_rep_rel_all_cell[i] for i in idx_uptake_included]



ref_poly_spline_all, ref_spline_derivative_all, ref_rel_t_exp_df_T_plot_all =fit_spline_condition(ref_rep_rel_all_cell)
times = np.arange(14, 47, .1)
spline_eval_ref_all = ref_poly_spline_all(times)
derivative_eval_ref_all = ref_spline_derivative_all(times)

# plot_spline_condition("", ref_poly_spline_all, "Reference all", data_points=True, pulse_rel_t_exp_df_T_plot=ref_rel_t_exp_df_T_plot_all)
plot_spline_derivative_condition("", ref_spline_derivative_all, "Reference all")


times = np.arange(14, 47, .1)
ref_poly_spline_p, ref_spline_derivative_p, ref_rel_t_exp_df_T_plot_p =fit_spline_condition(ref_pulse_time_series)
spline_eval_ref_p = ref_poly_spline_p(times)
derivative_eval_ref_p = ref_spline_derivative_p(times)

# plot_spline_condition("", ref_poly_spline_p, "Reference p", data_points=True, pulse_rel_t_exp_df_T_plot=ref_rel_t_exp_df_T_plot_p)
plot_spline_derivative_condition("", ref_spline_derivative_p, "Reference p")



### All pulse experimental conditions df saving

In [ ]:
### Save all selected pulses for experiment in one df, sorted by pulse condition number

repo_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/"
#3 obj [:5]
results_path_3obj = "Pulse_seed_pop_DsRED_inhibitor/ZF1_ZF2_only/Pulse_pop_DsRED_inhibitor_3obj_126h_ZF1_ZF2_seed_0/Results_analysis/"
file_name_3obj = "all_cell_metrics_low_t_pulse.csv"
file_name_3obj_ts = "all_cell_selected_results_low_t_pulse.csv"


#t pulse [1:2]
results_path_tpulse = "Pulse_seed_pop_DsRED_inhibitor/ZF1_ZF2_only/Pulse_pop_DsRED_inhibitor_t_pulse_126h_ZF1_ZF2_seed_0/Results_analysis_sub_opt/"
file_name_tpulse = "all_cell_metrics_sub_opt.csv"
file_name_tpulse_ts = "all_cell_selected_results_sub_opt.csv"

all_cell_df_tpulse = pd.read_csv(repo_path+results_path_tpulse+file_name_tpulse).drop(["Unnamed: 0", "Topology"], axis=1)
all_cell_df_tpulse_ts = pd.read_csv(repo_path+results_path_tpulse+file_name_tpulse_ts).drop(["Unnamed: 0", "Topology", "t_pulse_mean", "prominence_rel_mean"], axis=1)
all_cell_df_tpulse_ts_exp = all_cell_df_tpulse_ts.copy().iloc[1:2]
all_cell_df_tpulse_ts_exp.at[1, "Rep_rel time series for each cell"] = eval(all_cell_df_tpulse_ts_exp.at[1, "Rep_rel time series for each cell"])
all_cell_df_tpulse_ts_exp.at[1, "Rep_rel time series mean"] = eval(all_cell_df_tpulse_ts_exp.at[1, "Rep_rel time series mean"])

all_cell_df_tpulse_exp = all_cell_df_tpulse.copy().iloc[1:2]
all_cell_df_tpulse_exp["Edge_list"] = 0
all_cell_df_tpulse_exp["Doses"] = 0
all_cell_df_tpulse_exp["Doses"] = all_cell_df_tpulse_exp["Doses"].astype(object)
all_cell_df_tpulse_exp["Edge_list"] = all_cell_df_tpulse_exp["Edge_list"].astype(object)
all_cell_df_tpulse_exp.at[1, "Edge_list"] = [('P1', 'Z1'), ('Z1', 'Rep'), ('Z1', 'I2'), ('I2', 'Z1'), ('Z1', 'Z1'), ('I2', 'Rep')]
all_cell_df_tpulse_exp.at[1, "Doses"] = {'I2': 20, 'Z1': 50, 'Rep': 200}
# print(all_cell_df_tpulse_exp)

all_cell_df_tpulse_full = pd.concat([all_cell_df_tpulse_exp, all_cell_df_tpulse_ts_exp], axis=1)

### 3 obj
all_cell_df_3obj = pd.read_csv(repo_path+results_path_3obj+file_name_3obj).drop(["Unnamed: 0", "Topology"], axis=1)
all_cell_df_3obj_ts = pd.read_csv(repo_path+results_path_3obj+file_name_3obj_ts).drop(["Unnamed: 0", "Topology", "t_pulse_mean", "prominence_rel_mean", 'peak_rel_mean'], axis=1)
all_cell_df_3obj_ts_exp = all_cell_df_3obj_ts.copy().iloc[:5]

for index, row in all_cell_df_3obj_ts_exp.iterrows():
    all_cell_df_3obj_ts_exp.at[index, "Rep_rel time series for each cell"] = eval(row["Rep_rel time series for each cell"])
    all_cell_df_3obj_ts_exp.at[index, "Rep_rel time series mean"] = eval(row["Rep_rel time series mean"])

all_cell_df_3obj_exp = all_cell_df_3obj.copy().iloc[:5]
all_cell_df_3obj_exp["Edge_list"] = [0, 0, 0, 0, 0]
all_cell_df_3obj_exp["Doses"] = [0, 0, 0, 0, 0]
all_cell_df_3obj_exp["Doses"] = all_cell_df_3obj_exp["Doses"].astype(object)
all_cell_df_3obj_exp["Edge_list"] = all_cell_df_3obj_exp["Edge_list"].astype(object)
all_cell_df_3obj_exp.at[0, "Edge_list"] = [('P1', 'Z1'), ('Z1', 'Rep'), ('Z1', 'I2'), ('I2', 'Z1'), ('Z1', 'Z1'), ('I2', 'Rep')]
all_cell_df_3obj_exp.at[1, "Edge_list"] = [('P1', 'Z1'), ('Z1', 'Rep'), ('Z1', 'I2'), ('I2', 'Z1'), ('Z1', 'Z1'), ('I2', 'Rep')]
all_cell_df_3obj_exp.at[2, "Edge_list"] = [('P1', 'Z1'), ('Z1', 'Rep'), ('Z1', 'I2'), ('I2', 'Z1'), ('Z1', 'Z1'), ('I2', 'Rep')]
all_cell_df_3obj_exp.at[3, "Edge_list"] = [('P1', 'Z1'), ('Z1', 'Rep'), ('Z1', 'I2'), ('I2', 'Z1'), ('Z1', 'Z1'), ('I2', 'Rep')]
all_cell_df_3obj_exp.at[4, "Edge_list"] = [('P1', 'Z1'), ('Z1', 'Rep'), ('Z1', 'I2'), ('I2', 'Z1'), ('Z1', 'Z1'), ('I2', 'Rep')]
all_cell_df_3obj_exp.at[0, "Doses"] = {'I2': 25, 'Z1': 75, 'Rep': 200}
all_cell_df_3obj_exp.at[1, "Doses"] = {'I2': 25, 'Z1': 70, 'Rep': 200}
all_cell_df_3obj_exp.at[2, "Doses"] = {'I2': 25, 'Z1': 65, 'Rep': 200}
all_cell_df_3obj_exp.at[3, "Doses"] = {'I2': 25, 'Z1': 65, 'Rep': 200}
all_cell_df_3obj_exp.at[4, "Doses"] = {'I2': 30, 'Z1': 75, 'Rep': 200}

all_cell_df_3obj_full = pd.concat([all_cell_df_3obj_exp, all_cell_df_3obj_ts_exp], axis=1)

all_cell_df_exp = pd.concat([all_cell_df_3obj_full, all_cell_df_tpulse_full], ignore_index=True)
all_cell_df_exp["pulse exp condition"] = [1, 2, 4, 6, 5, 3]
all_cell_df_exp.sort_values("pulse exp condition", inplace=True, ignore_index=True)
print(all_cell_df_exp)

path_save = "Pulse_seed_pop_DsRED_inhibitor/ZF1_ZF2_only/"
all_cell_df_exp.to_csv(repo_path+path_save+"pulse_conditions_exp_new.csv")
import pickle
with open(repo_path+path_save + "pulse_conditions_exp.pkl", "wb"
    ) as fid:
        pickle.dump(all_cell_df_exp, fid)

### Pulse circuit simulations with exp conditions

In [ ]:
import pickle
repo_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/"
path_pulses = "Pulse_seed_pop_DsRED_inhibitor/ZF1_ZF2_only/pulse_conditions_exp.pkl"

pulse_conditions = pd.read_pickle(repo_path+path_pulses)
# display(pulse_conditions)
# print(pulse_conditions.at[0, "Rep_rel time series mean"])


### Compare between 20 cell and 200 cell models

In [ ]:
from pulse_generator_problem import PulseGenerator
from define_circuit import Topo

all_t_pulse_20 = pulse_conditions["t_pulse_mean"].tolist()
all_prom_rel_20 = pulse_conditions["prominence_rel_mean"].tolist()
all_t_pulse_200 = []
all_prom_rel_200 = []
all_t_pulse_2000 = []
all_prom_rel_2000 = []
all_cells_dict_list_200 = []
all_cells_dict_list_2000 = []
peak_rel_gmeans = []
prom_rel_gmeans = []
t_pulse_gmeans = []
for index, row in pulse_conditions.iterrows():
    print("20 cell t_pulse, prominence_rel: ", all_t_pulse_20[index], all_prom_rel_20[index])
    topo = Topo(pulse_conditions.at[index, "Edge_list"], pulse_conditions.at[index, "Doses"], "P1")

    pulse_exp_200 = PulseGenerator(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                            DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                            probability_mutation=None, mutate_dose=True, pop=True, Z_mat = Z_200, num_processes=8, max_time=126, single_cell_tracking=True)
    [[t_pulse_200, prominence_rel_200], all_cells_dict_200] = pulse_exp_200.func(topo)
    print("200 cell t_pulse, prominence_rel: ", t_pulse_200, prominence_rel_200)
    all_t_pulse_200.append(t_pulse_200)
    all_prom_rel_200.append(prominence_rel_200)
    all_cells_dict_list_200.append(all_cells_dict_200)
    all_cells_ts = all_cells_dict_200["Rep_rel time series for each cell"]

    # for cell_ts in all_cells_ts:
    #     plt.plot(np.arange(0, 42, 1), cell_ts[:42])
    # plt.plot(np.arange(0, 127, 1), rep_rel_200_gmean, color="k")
    # plt.show()
    print("***** topology ", index, " done *****")
# print("peak_rels: ", peak_rel_list)
# print("prom_rels: ", prom_rel_list)
# print("t_pulses:", t_pulse_list)

all_cell_results_df = pd.DataFrame.from_dict(all_cells_dict_list_200)
all_cell_results_df["single_cell_peaks"] = 0
all_cell_results_df["single_cell_peaks"] = all_cell_results_df["single_cell_peaks"].astype(object)
all_cell_results_df["single_cell_prominence"] = 0
all_cell_results_df["single_cell_prominence"] = all_cell_results_df["single_cell_prominence"].astype(object)
for index, row in all_cell_results_df.iterrows():
    peak_cell_list = []
    prom_cell_list = []
    for i in range(200):
        peak_cell = max(row["Rep_rel time series for each cell"][i])
        peak_cell_list.append(peak_cell)
        prom_cell =  pulse_exp_200.calc_prominence_rel(row["Rep_rel time series for each cell"][i], peak_cell)
        prom_cell_list.append(prom_cell)
    all_cell_results_df.at[index, "single_cell_peaks"] = peak_cell_list
    all_cell_results_df.at[index, "single_cell_prominence"] = prom_cell_list

# display(all_cell_results_df)
all_cell_results_df.to_pickle("/Users/kdreyer/Desktop/200_cell_all_cell_results_df.pkl")

# pulse_pop_comparison = {"20_cell_t_pulse": all_t_pulse_20, "20_cell_prom_rel": all_prom_rel_20,
#                      "200_cell_t_pulse": all_t_pulse_200, "200_cell_prom_rel": all_prom_rel_200,
#                      "2000_cell_t_pulse": all_t_pulse_2000, "2000_cell_prom_rel": all_prom_rel_2000,
# }
# pulse_pop_df = pd.DataFrame(pulse_pop_comparison)
# pulse_pop_df_pos = pulse_pop_df.abs()
# path_save = "Pulse_seed_pop_DsRED_inhibitor/ZF1_ZF2_only/"
# pulse_pop_df_pos.to_csv(repo_path + path_save+"pop_model_comparisons.csv")

# filename200 = repo_path + path_save+"200_cell_all_cells_dicts.pkl"
# with open(filename200, "wb") as fid:
#     pickle.dump(all_cells_dict_list_200, fid)

# filename2000 = repo_path + path_save+"2000_cell_all_cells_dicts.pkl"
# with open(filename2000, "wb") as fid:
#     pickle.dump(all_cells_dict_list_2000, fid)


# for cell in range(200):
#     time_series = all_cells_dict['Rep_rel time series for each cell'][cell]
#     plt.plot(np.arange(0, 127, 1), time_series, label=str(cell))
# plt.ylim([0, 5])
# plt.plot(np.arange(0, 127, 1), all_cells_dict['Rep_rel time series mean'], color="k")
# plt.xlim([0, 46])
# plt.legend()
# plt.show()
# print(t_pulse, prominence_rel_neg)

### Looking at outliers from simulations with 200 and 2000 cell models

In [ ]:
no_pulses = []
no_pulse_cell_nums = []
no_pulse_cell_maxes = []
pulses = []
pulse_cell_nums = []
pulse_cell_maxes = []
for cell in range(2000):
    time_series = all_cells_dict_list_2000[0]['Rep_rel time series for each cell'][cell]
    if max(time_series) > time_series[-1]:
        pulses.append(time_series)
        pulse_cell_nums.append(cell)
        pulse_cell_maxes.append(max(time_series))
    else:
        if max(time_series) > 100:
            print("no pulse outlier max: ", max(time_series), " cell: ", cell)
        no_pulses.append(time_series)
        no_pulse_cell_nums.append(cell)
        no_pulse_cell_maxes.append(max(time_series))

# plt.plot(np.arange(0, 127, 1), all_cells_dict['Rep_rel time series mean'], color="k")
plt.figure(figsize=(3,3))
for i, pulse in enumerate(pulses):
    # print(pulse_cell_nums[i])
    # print(pulse_cell_maxes[i])
    plt.plot(np.arange(0, 127, 1), pulse)
    plt.xlim([0, 46])
    plt.xlabel("time (h)")
    plt.ylabel("Reporter_rel")
    plt.title("2000 cell model pulse time series,\n" + str(len(pulses)) + " pulses")
    # plt.legend()
    # plt.ylim([0, 20])
# plt.show()
plt.savefig(repo_path + path_save+"2000_cell_pulse_time_series_1.svg")

plt.figure(figsize=(3,3))
for i, no_pulse in enumerate(no_pulses):
    print(no_pulse_cell_nums[i])
    print(no_pulse_cell_maxes[i])
    print("******")
    plt.plot(np.arange(0, 127, 1), no_pulse)
    plt.xlim([0, 46])
    plt.xlabel("time (h)")
    plt.ylabel("Reporter_rel")
# plt.show()
plt.savefig(repo_path + path_save+"2000_cell_no_pulse_time_series_1.svg")


In [ ]:
path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Pulse_pop/Experiment_synTF1&2/200_cell_all_cell_results_df.pkl"
all_cells_200 = pd.read_pickle(path)
# display(all_cells_200)
pulse_1_200 = all_cells_200.iloc[0]
# display(pulse_1_200['Rep_rel time series for each cell'])
fig, ax = plt.subplots(1, 1, figsize=(2, 2))
for cell in range(200):
    time_series = pulse_1_200['Rep_rel time series for each cell'][cell]
    if max(time_series) > 5:
    #     color_ = "k"
    # else:
    #     color_ = "grey"
        plt.plot(np.arange(0, 127, 1), time_series, label=str(cell), color="grey")
plt.ylim([0, 35])
plt.plot(np.arange(0, 127, 1), pulse_1_200['Rep_rel time series mean'], color="k", linewidth="2")
plt.xlim([0, 46])
plt.ylim(bottom=0)
plt.xlabel("time (h)")
plt.ylabel("Rep_rel")
plt.yticks([0, 10, 20, 30])
# plt.legend()
# plt.show()
plt.savefig('/Users/kdreyer/Desktop/"200_cell_pulse_ts.svg')

In [ ]:
path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Pulse_pop/Experiment_synTF1&2/pulse_conditions_exp.pkl"
all_cells_20 = pd.read_pickle(path)
# display(all_cells_20)
fig, ax = plt.subplots(1, 1, figsize=(2, 2))
for cell in range(20):
    time_series = all_cells_20.at[0,'Rep_rel time series for each cell'][cell]

    plt.plot(np.arange(0, 127, 1), time_series, label=str(cell), color="grey")
plt.ylim([0, 35])
plt.plot(np.arange(0, 127, 1), all_cells_20.at[0, 'Rep_rel time series mean'], color="k", linewidth="2")
plt.xlim([0, 46])
plt.ylim(bottom=0)
plt.xlabel("time (h)")
plt.ylabel("Rep_rel")
plt.yticks([0, 10, 20, 30])
# plt.legend()
# plt.show()
plt.savefig('/Users/kdreyer/Desktop/"20_cell_pulse_ts.svg')

In [ ]:
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from pymoo.indicators.hv import HV
from rankcrowding import RankAndCrowding
nds = RankAndCrowding()
path = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/outdated/2024-10-07_Pulse_pop_DsRED_t_pulse_ZF1_ZF2_fullpop/unique_objectives_df.pkl"
pulse_obj = pd.read_pickle(path)
obj_arr = pulse_obj.to_numpy()
# print(obj_arr)
S = nds.do(obj_arr, 168)
obj = obj_arr[S]
# print(obj)
obj_df = pd.DataFrame(obj, columns=["t_pulse", "prom_rel"])
display(obj_df[obj_df["t_pulse"] <= 24])

In [ ]:
import pickle
repo_path = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/"
path_pulses = "2024-09-11_Pulse_pop_DsRED_inhibitor_3_obj_80gens_gmean_seed_0_seed_0/2024-09-12_results_analysis/selected_results_full_pareto_front.pkl"

pulse_conditions = pd.read_pickle(repo_path+path_pulses)
pulse_conditions.head()

In [ ]:

all_t_pulse_20 = pulse_conditions["t_pulse"].tolist()
all_peak_rel_20 = pulse_conditions["peak_rel"].tolist()
all_prom_rel_20 = pulse_conditions["prominence_rel"].tolist()
all_t_pulse_200 = []
all_peak_rel_200 = []
all_prom_rel_200 = []


for index, row in pulse_conditions.tail(n=11).iterrows():
    print("20 cell t_pulse, peak_rel, prominence_rel: ", all_t_pulse_20[index], all_peak_rel_20[index], all_prom_rel_20[index])
    topo = row["Topology"]

    pulse_exp_200 = PulseGenerator(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                            DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                            probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200,
                            num_processes=8, obj_labels= ["t_pulse", "peak_rel", "prominence_rel"], max_time=126,
                            single_cell_tracking=True)
    [[t_pulse_200, peak_rel_200, prominence_rel_200], all_cells_dict_200] = pulse_exp_200.func(topo)
    print("200 cell t_pulse, peak_rel, prominence_rel: ", t_pulse_200, peak_rel_200, prominence_rel_200)
    all_t_pulse_200.append(t_pulse_200)
    all_peak_rel_200.append(peak_rel_200)
    all_prom_rel_200.append(prominence_rel_200)
    all_cells_dict_list_200.append(all_cells_dict_200)
    all_cells_ts = all_cells_dict_200["Rep_rel time series for each cell"]

    # pulse_exp_2000 = PulseGenerator(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
    #                         DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
    #                         probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_2000, num_processes=8, max_time=126, single_cell_tracking=True)
    # [[t_pulse_2000, prominence_rel_2000], all_cells_dict_2000] = pulse_exp_2000.func(topo)
    # print("2000 cell t_pulse, prominence_rel: ", t_pulse_2000, prominence_rel_2000)
    # all_t_pulse_2000.append(t_pulse_2000)
    # all_prom_rel_2000.append(prominence_rel_2000)
    # all_cells_dict_list_2000.append(all_cells_dict_2000)
    # for cell_ts in all_cells_ts:
    #     plt.plot(np.arange(0, 42, 1), cell_ts[:42])

    path_save = "2024-09-11_Pulse_pop_DsRED_inhibitor_3_obj_80gens_gmean_seed_0_seed_0/2024-09-12_results_analysis/"

    if prominence_rel_200 != 0:
        plt.figure(figsize=(3,3))
        for cell in range(200):
            plt.plot(np.arange(0, 127, 1), all_cells_ts[cell])
        plt.plot(np.arange(0, 127, 1), all_cells_dict_200["Rep_rel time series mean"], color="k")
        plt.savefig(repo_path+ path_save + "all_cell_ts_200_"+str(index)+".svg")
    # plt.show()
    print("***** topology ", index, " done *****")
# print("peak_rels: ", peak_rel_list)
# print("prom_rels: ", prom_rel_list)
# print("t_pulses:", t_pulse_list)


# pulse_pop_comparison = {"20_cell_t_pulse": all_t_pulse_20, "20_cell_peak_rel": all_peak_rel_20,
#                         "20_cell_prom_rel": all_prom_rel_20,
#                      "200_cell_t_pulse": all_t_pulse_200, "200_cell_peal_rel": all_peak_rel_200,
#                      "200_cell_prom_rel": all_prom_rel_200
# }
# pulse_pop_df = pd.DataFrame(pulse_pop_comparison)
# pulse_pop_df_pos = pulse_pop_df.abs()
# path_save = "2024-09-11_Pulse_pop_DsRED_inhibitor_3_obj_80gens_gmean_seed_0_seed_0/2024-09-12_results_analysis/"
# pulse_pop_df_pos.to_csv(repo_path + path_save+"pop_model_comparisons.csv")

# filename200 = repo_path + path_save+"200_cell_all_cells_dicts.pkl"
# with open(filename200, "wb") as fid:
#     pickle.dump(all_cells_dict_list_200, fid)

In [ ]:
path_results = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/Pulse_seed_pop_DsRED_inhibitor/gmean/"
path_file = "2024-09-11_Pulse_pop_DsRED_inhibitor_3_obj_80gens_gmean_seed_0/2024-09-12_results_analysis/" 
fname = "pop_model_comparisons_gmean.csv"
pulse_comparisons = pd.read_csv(path_results+path_file+fname)
# display(pulse_comparisons)
plt.figure(figsize=(2.25, 2.25))
plt.plot(pulse_comparisons["20_cell_prom_rel"], pulse_comparisons["200_cell_prom_rel"], linestyle="none", marker="o", markersize="2", color="k")
plt.plot([0, 1.5], [0, 1.5], linestyle="dashed", color="grey")
plt.xticks([0, 0.5, 1, 1.5])
plt.yticks([0, 0.5, 1, 1.5])
plt.xlabel("20-cell prominence_rel")
plt.ylabel("200-cell prominence_rel")
# plt.show()
plt.savefig(path_results+path_file+"20v200_parity_plot.svg")

In [ ]:
path_results = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/Pulse_seed_pop_DsRED_inhibitor/frac_pulse/"
path_file = "2024-09-11_Pulse_pop_DsRED_inhibitor_frac_pulse_seed_0/2024-09-12_results_analysis/" 
fname = "pop_model_comparisons.csv"
pulse_comparisons = pd.read_csv(path_results+path_file+fname)
# display(pulse_comparisons)
# plt.figure(figsize=(2.25, 2.25))
# plt.plot(pulse_comparisons["20_cell_prom_rel"], pulse_comparisons["200_cell_prom_rel"], linestyle="none", marker="o", markersize="2", color="k")
# plt.plot([0, 60], [0, 60], linestyle="dashed", color="grey")
# plt.xticks([0, 20, 40, 60])
# plt.yticks([0, 20, 40, 60])
# plt.xlabel("20-cell prominence_rel")
# plt.ylabel("200-cell prominence_rel")
# # plt.show()
# plt.savefig(path_results+path_file+"20v200_parity_plot_prom.svg")

plt.figure(figsize=(2.25, 2.25))
plt.plot(pulse_comparisons["20_cell_frac_pulse"], pulse_comparisons["200_cell_frac_pulse"], linestyle="none", marker="o", markersize="2", color="k")
plt.plot([0, 1], [0, 1], linestyle="dashed", color="grey")
# plt.xticks([0, 20, 40, 60])
# plt.yticks([0, 20, 40, 60])
plt.xlabel("20-cell frac_pulse")
plt.ylabel("200-cell frac_pulse")
# plt.show()
plt.savefig(path_results+path_file+"20v200_parity_plot_frac.svg")

### Pulse 20 vs. 200 cell model comparisons for frac_pulse GA run

In [ ]:
import pickle
repo_path = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/"
path_frac_pulses = "2024-09-11_Pulse_pop_DsRED_inhibitor_frac_pulse_seed_0_seed_0/2024-09-12_results_analysis/selected_results_full_pareto_front.pkl"

pulse_conditions = pd.read_pickle(repo_path+path_frac_pulses)
pulse_conditions.head()

In [ ]:
from pulse_generator_problem import PulseGenerator
# all_t_pulse_20 = pulse_conditions["t_pulse"].tolist()
# all_peak_rel_20 = pulse_conditions["peak_rel"].tolist()
all_prom_rel_20 = pulse_conditions["prominence_rel"].tolist()
all_frac_pulse_20 = pulse_conditions["frac_pulse"].tolist()
# all_t_pulse_200 = []
# all_peak_rel_200 = []
all_prom_rel_200 = []
all_frac_pulse_200 = []
all_cells_dict_list_200 = []

for index, row in pulse_conditions.iterrows():
    print("20 cell prominence_rel, frac_pulse: ", all_prom_rel_20[index], all_frac_pulse_20[index])
    topo = row["Topology"]

    pulse_200 = PulseGenerator(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                            DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                            probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200,
                            num_processes=8, obj_labels= ["frac_pulse", "prominence_rel"], max_time=126,
                            single_cell_tracking=True)
    [[frac_pulse_200, pulse_prom_rel_200], all_cells_dict_200] = pulse_200.func(topo)
    print("200 cell prominence_rel, frac_pulse: ", pulse_prom_rel_200, frac_pulse_200)
    all_frac_pulse_200.append(frac_pulse_200)
    all_prom_rel_200.append(pulse_prom_rel_200)
    all_cells_dict_list_200.append(all_cells_dict_200)
    all_cells_ts = all_cells_dict_200["Rep_rel time series for each cell"]

    # pulse_exp_2000 = PulseGenerator(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
    #                         DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
    #                         probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_2000, num_processes=8, max_time=126, single_cell_tracking=True)
    # [[t_pulse_2000, prominence_rel_2000], all_cells_dict_2000] = pulse_exp_2000.func(topo)
    # print("2000 cell t_pulse, prominence_rel: ", t_pulse_2000, prominence_rel_2000)
    # all_t_pulse_2000.append(t_pulse_2000)
    # all_prom_rel_2000.append(prominence_rel_2000)
    # all_cells_dict_list_2000.append(all_cells_dict_2000)
    # for cell_ts in all_cells_ts:
    #     plt.plot(np.arange(0, 42, 1), cell_ts[:42])

    path_save = "2024-09-11_Pulse_pop_DsRED_inhibitor_frac_pulse_seed_0_seed_0/2024-09-12_results_analysis/"

    plt.figure(figsize=(3,3))
    for cell in range(200):
        plt.plot(np.arange(0, 127, 1), all_cells_ts[cell])
    plt.plot(np.arange(0, 127, 1), all_cells_dict_200["Rep_rel time series mean"], color="k")
    plt.savefig(repo_path+ path_save + "all_cell_ts_200_"+str(index)+".svg")
    # plt.show()
    print("***** topology ", index, " done *****")

pulse_pop_comparison = {
                        "20_cell_prom_rel": all_prom_rel_20,
                        "20_cell_frac_pulse": all_frac_pulse_20,
                     "200_cell_prom_rel": all_prom_rel_200,
                     "200_cell_frac_pulse": all_frac_pulse_200,
                     
}
pulse_pop_df = pd.DataFrame(pulse_pop_comparison)
pulse_pop_df_pos = pulse_pop_df.abs()
path_save = "2024-09-11_Pulse_pop_DsRED_inhibitor_frac_pulse_seed_0_seed_0/2024-09-12_results_analysis/"
pulse_pop_df_pos.to_csv(repo_path + path_save+"pop_model_comparisons.csv")

filename200 = repo_path + path_save+"200_cell_all_cells_dicts.pkl"
with open(filename200, "wb") as fid:
    pickle.dump(all_cells_dict_list_200, fid)

### 20 vs 200 cell model for opt single cell topologies

In [ ]:
from pulse_generator_problem import PulseGenerator
from define_circuit import Topo

# pulse_pareto_single_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Pulse_single_cell/Optimized_hyperparams/t_pulse/Optimized_hyperparams_fixed_pop_max_hv/2024-10-09_Pulse_single_DsRED_t_pulse_opt_hps_ngen80_seed_5/2024-10-11_results_analysis/selected_results_full_pareto.pkl"
# pulse_pareto_single = pd.read_pickle(pulse_pareto_single_path)
pulse_single_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Pulse_single_cell/Optimized_hyperparams/t_pulse/Optimized_hyperparams_fixed_pop_max_hv/2024-10-09_Pulse_single_DsRED_t_pulse_opt_hps_ngen80_seed_5/"

pulse_selected = pd.read_pickle(pulse_single_path+"selected_unique_results.pkl")
# display(pulse_pareto_single)

all_t_pulse_20 = []
all_prom_rel_20 = []
all_cells_dict_list_20 = []

all_t_pulse_200 = []
all_prom_rel_200 = []
all_cells_dict_list_200 = []


for index, row in pulse_selected.iterrows():
    topo = row["topology"]

    pulse_20 = PulseGenerator(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                            DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                            probability_mutation=None, mutate_dose=True, pop=True, num_processes=8, max_time=126, single_cell_tracking=True)
    [[t_pulse_20, prominence_rel_20], all_cells_dict_20] = pulse_20.func(topo)
    if prominence_rel_20 != 0:
        print("20 cell t_pulse, prominence_rel: ", t_pulse_20, -prominence_rel_20)
    else:
        print("not a 20 cell pulse")
    all_t_pulse_20.append(t_pulse_20)
    all_prom_rel_20.append(prominence_rel_20)
    all_cells_dict_list_20.append(all_cells_dict_20)
    all_cells_ts = all_cells_dict_20["Rep_rel time series for each cell"]



    pulse_200 = PulseGenerator(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                            DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                            probability_mutation=None, mutate_dose=True, pop=True, Z_mat = Z_200, num_processes=8, max_time=126, single_cell_tracking=True)
    [[t_pulse_200, prominence_rel_200], all_cells_dict_200] = pulse_200.func(topo)
    if prominence_rel_200 != 0:
        print("200 cell t_pulse, prominence_rel: ", t_pulse_200, -prominence_rel_200)
    else:
        print("not a 200 cell pulse")
    all_t_pulse_200.append(t_pulse_200)
    all_prom_rel_200.append(prominence_rel_200)
    all_cells_dict_list_200.append(all_cells_dict_200)
    all_cells_ts = all_cells_dict_200["Rep_rel time series for each cell"]
    print("topology", index, " done")

plt.plot([abs(i) for i in all_prom_rel_20], [abs(i) for i in all_prom_rel_200], linestyle="none", marker="o")
plt.plot([0, 6], [0, 6], linestyle="dashed")
plt.xlabel("20 cell prom_rel")
plt.ylabel("200 cell prom_rel")
plt.show()

all_cell_results_df_200 = pd.DataFrame.from_dict(all_cells_dict_list_200)
all_cell_results_df_200["single_cell_peaks"] = 0
all_cell_results_df_200["single_cell_peaks"] = all_cell_results_df_200["single_cell_peaks"].astype(object)
all_cell_results_df_200["single_cell_prominence"] = 0
all_cell_results_df_200["single_cell_prominence"] = all_cell_results_df_200["single_cell_prominence"].astype(object)
for index, row in all_cell_results_df_200.iterrows():
    peak_cell_list = []
    prom_cell_list = []
    for i in range(200):
        peak_cell = max(row["Rep_rel time series for each cell"][i])
        peak_cell_list.append(peak_cell)
        prom_cell =  pulse_200.calc_prominence_rel(row["Rep_rel time series for each cell"][i], peak_cell)
        prom_cell_list.append(prom_cell)
    all_cell_results_df_200.at[index, "single_cell_peaks"] = peak_cell_list
    all_cell_results_df_200.at[index, "single_cell_prominence"] = prom_cell_list

# display(all_cell_results_df)
all_cell_results_df_200.to_pickle("/Users/kdreyer/Desktop/200_cell_subopt_results_df.pkl")


all_cell_results_df_20 = pd.DataFrame.from_dict(all_cells_dict_list_20)
all_cell_results_df_20["single_cell_peaks"] = 0
all_cell_results_df_20["single_cell_peaks"] = all_cell_results_df_20["single_cell_peaks"].astype(object)
all_cell_results_df_20["single_cell_prominence"] = 0
all_cell_results_df_20["single_cell_prominence"] = all_cell_results_df_20["single_cell_prominence"].astype(object)
for index, row in all_cell_results_df_20.iterrows():
    peak_cell_list = []
    prom_cell_list = []
    for i in range(20):
        peak_cell = max(row["Rep_rel time series for each cell"][i])
        peak_cell_list.append(peak_cell)
        prom_cell =  pulse_20.calc_prominence_rel(row["Rep_rel time series for each cell"][i], peak_cell)
        prom_cell_list.append(prom_cell)
    all_cell_results_df_20.at[index, "single_cell_peaks"] = peak_cell_list
    all_cell_results_df_20.at[index, "single_cell_prominence"] = prom_cell_list

# display(all_cell_results_df)
# all_cell_results_df_20.to_pickle("/Users/kdreyer/Desktop/20_cell_subopt_results_df.pkl")


# pulse_pop_comparison = {"20_cell_t_pulse": all_t_pulse_20, "20_cell_prom_rel": all_prom_rel_20,
#                      "200_cell_t_pulse": all_t_pulse_200, "200_cell_prom_rel": all_prom_rel_200,
#                      "2000_cell_t_pulse": all_t_pulse_2000, "2000_cell_prom_rel": all_prom_rel_2000,
# }
# pulse_pop_df = pd.DataFrame(pulse_pop_comparison)
# pulse_pop_df_pos = pulse_pop_df.abs()
# path_save = "Pulse_seed_pop_DsRED_inhibitor/ZF1_ZF2_only/"
# pulse_pop_df_pos.to_csv(repo_path + path_save+"pop_model_comparisons.csv")

# filename200 = repo_path + path_save+"200_cell_all_cells_dicts.pkl"
# with open(filename200, "wb") as fid:
#     pickle.dump(all_cells_dict_list_200, fid)

In [ ]:
plt.plot([abs(i) for i in all_prom_rel_20], [abs(i) for i in all_prom_rel_200], linestyle="none", marker="o", markersize="1")
plt.plot([0, 1], [0, 1], linestyle="dashed")
plt.xlabel("20 cell prom_rel")
plt.ylabel("200 cell prom_rel")
plt.show()

In [ ]:
all_cell_results_df_20 = pd.read_pickle("/Users/kdreyer/Desktop/20_cell_all_cell_results_df_opt.pkl")
all_cell_results_df_200 = pd.read_pickle("/Users/kdreyer/Desktop/200_cell_all_cell_results_df_opt.pkl")
# display(all_cell_results_df_200)
prom_rel_200_vals = []
prom_rel_20_vals = []
for index, row in all_cell_results_df_200.iterrows():
    peak_rel_200 = pulse_200.calc_peak_rel(row["Rep_rel time series mean"])
    prom_rel_200 = pulse_200.calc_prominence_rel(row["Rep_rel time series mean"], peak_rel_200)
    prom_rel_200_vals.append(prom_rel_200)

    peak_rel_20 = pulse_20.calc_peak_rel(all_cell_results_df_20.loc[index, "Rep_rel time series mean"])
    prom_rel_20 = pulse_20.calc_prominence_rel(all_cell_results_df_20.loc[index, "Rep_rel time series mean"], peak_rel_20)
    prom_rel_20_vals.append(prom_rel_20)
    # if prom_rel_20 != 0 and prom_rel_200 == 0:
    #     t_pulse = pulse_20.calc_t_pulse(np.arange(0, 127, 1), all_cell_results_df_20.loc[index, "Rep_rel time series mean"], peak_rel_20, prom_rel_20)
#         print(t_pulse)
plt.plot(prom_rel_20_vals, prom_rel_200_vals, linestyle="none", marker="o")
plt.plot([0, 6], [0, 6])
plt.show()


In [ ]:
# plt.plot([abs(i) for i in all_prom_rel_20], [abs(i) for i in all_prom_rel_200], linestyle="none", marker="o")
# plt.plot([0, 6], [0, 6], linestyle="dashed")
# plt.xlabel("20 cell prom_rel")
# plt.ylabel("200 cell prom_rel")
# plt.show()
pulse_single_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Pulse_single_cell/Optimized_hyperparams/t_pulse/Optimized_hyperparams_fixed_pop_max_hv/2024-10-09_Pulse_single_DsRED_t_pulse_opt_hps_ngen80_seed_5/"
all_obj_fname = "all_objectives.pkl"
all_circuits_fname = "all_circuits.pkl"
all_obj = pd.read_pickle(pulse_single_path+all_obj_fname)
with open(pulse_single_path+all_circuits_fname, "rb") as fid:
    all_circuits = pickle.load(fid)
all_obj = abs(all_obj)
# print(all_obj)
unique_obj, unique_indices = np.unique(all_obj, axis=0,
                                return_index=True)
unique_obj_df = pd.DataFrame(unique_obj, columns=["t_pulse","prominence_rel"])
select_subopt = unique_obj_df.copy()
select_subopt = select_subopt[select_subopt["prominence_rel"] >=5] 
select_circuits = all_circuits[select_subopt.index].flatten()
select_subopt["topology"] = select_circuits
# print(select_circuits)
# display(select_subopt)
print(select_subopt[select_subopt["t_pulse"] > 55])
# plt.plot(unique_obj[:, 0], unique_obj[:, 1], linestyle="none", marker="o")
# plt.plot(pulse_pareto_single["t_pulse"], pulse_pareto_single["prominence_rel"], linestyle="none", marker="o")
# plt.plot([i-2 for i in pulse_pareto_single["t_pulse"].tolist()], [i-2 for i in pulse_pareto_single["prominence_rel"].tolist()])
# plt.xlim(0, 20)
# plt.ylim(bottom=5)
# unique_circuits = all_circuits[unique_indices]
# select_subopt.to_pickle(pulse_single_path+"selected_unique_results.pkl")


# Test

In [ ]:
from pymoo.indicators.hv import HV
ref_point = np.array([126, 0])
hv = HV(ref_point=ref_point)

pulse_combo_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Combinatorial_search/pulse_tpulse_pareto_dsred1_final.pkl"
path_GA = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Pulse_single_cell/Optimized_hyperparams/t_pulse/Optimized_hyperparams_fixed_pop_max_hv/2024-10-09_Pulse_single_DsRED_t_pulse_opt_hps_ngen80_seed_5/final_objectives_df.pkl"
pulse_pareto = pd.read_pickle(pulse_combo_path)
print(pulse_pareto)
hv_combo = hv(pulse_pareto)
print(hv_combo)
print(len(pulse_pareto))
pulse_GA = pd.read_pickle(path_GA)
display(pulse_GA.drop_duplicates())

plt.plot(pulse_pareto[:,0], abs(pulse_pareto[:, 1]), linestyle="none", marker="o")
plt.plot(pulse_GA["t_pulse"], abs(pulse_GA["prominence_rel"]), linestyle="none", marker="o", fillstyle="none")